In [ ]:
!pip install --upgrade pip
!pip install transformers==4.40.1 datasets==2.19.0 sentence-transformers==2.7.0 faiss-cpu==1.8.0 llama-index==0.10.34 llama-index-embeddings-huggingface==0.2.0 -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.0 MB/s eta

# **문장 임베딩을 활용한 단어 간 유사도 계산**

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

smodel = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
dense_embeddings = smodel.encode(['학교', '공부', '운동'])
cosine_similarity(dense_embeddings) #코사인 유사도
# array([[1.0000001 , 0.5950744 , 0.32537547],
#       [0.5950744 , 1.0000002 , 0.54595673],
#       [0.32537547, 0.54595673, 0.99999976]], dtype=float32)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


array([[0.99999994, 0.59507436, 0.32537556],
       [0.59507436, 1.        , 0.54595697],
       [0.32537556, 0.54595697, 0.99999994]], dtype=float32)

# **원핫 인코딩의 한계**

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

word_dict = {"school": np.array([[1, 0, 0]]), "study": np.array([[0, 1, 0]]), "workout": np.array([[0, 0, 1]])}

# 두 단어 사이의 코사인 유사도 계산하기
# cosine_similarity(A, B) -> 코사인 유사도 측정 메서드
cosine_school_study = cosine_similarity(word_dict['school'], word_dict['study']) # 0
cosine_school_workout = cosine_similarity(word_dict['school'], word_dict['workout']) # 0

# **Sentence-Transformers 라이브러리로 바이 인코더 생성하기**

In [ ]:
# 문장을 벡터로 변환하는 라이브러리
from sentence_transformers import SentenceTransformer, models

# 사용할 BERT 모델
# 허깅페이스에서 임베딩 모델 불러오기
word_embedding_model = models.Transformer('klue/roberta-base')
# 풀링 층 차원 입력하기 -> 단어별 임베딩을 하나의 문장 임베딩으로 압축
# get_word_embedding_dimension() -> 모델의 임베딩 크기(차원 수)를 가져옴
# 단어 차원의 수를 가져와서 문장 벡터의 차원으로 설정
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# 두 모듈 결합하기
# 임베딩 모델과 풀링층 결합
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/375 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/752k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

# **코드로 살펴보는 평균 모드**

In [ ]:
def mean_pooling(model_output, attention_mask):
  # model_output -> 언어 모델 출력
  # model_output[0] -> 출력 중 마지막 층의 출력만 사용 -> 마지막 층에서 나온 토큰별 임베딩(벡터) -> 모든 처리를 거쳐 [0]에 저장되기 때문
  # 0은 출력 리스트의 첫 번째 요소를 의미하는 것이지 첫 번째 층의 출력을 의미하는게 아님
  token_embeddings = model_output[0]
  # unsqueeze(-1) -> 마지막 차원 추가
  # attention_mask는 [batch_size, num_tokens] 형태라서 token_embeddings(batch_size, num_tokens, hidden_dim)와 곱할 수 없다

  # expand(token_embeddings.size()) -> 텐서 차원 확장 -> token_embeddings.size()와 같게

  # .float() -> 벡터와 곱하기 위해 형변환
  input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()

  # attention_mask -> 미래 마스크가 아닌 패딩 마스크
  # 패딩 토큰이 학습과 계산에 영향을 주지 않도록 무시하는 역할
  # BERT 같은 인코더 기반 모델에서 사용

  # 패딩이 0이 아닐수도 있어서 패딩 마스크와 곱해 패딩 부분의 값을 0으로 만든다
  # sum(x, 1) -> 두번째 차원을 따라 더한다
  sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)

  # input_mask_expanded.sum(1) -> 패딩 마스크의 두 번째 차원을 더해 단어 개수를 구한다
  # input_mask_expanded =
    # [[1, 1, 1, 0, 0],  # 첫 번째 문장 (5개 토큰 중 3개가 실제 단어)
    # [1, 1, 1, 1, 1]]  # 두 번째 문장 (모든 토큰이 실제 단어)

  # torch.clamp -> min 혹은 max의 범주에 해당하도록 값 변경
  # min=1e-9 -> 최솟값이 해당 값보다 작으면 해당 값으로 변경
  # 0으로 나누는 것을 방지하기 위함
  sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)

  return sum_embeddings / sum_mask

  # 1. 모델 아웃풋과 패딩 마스크를 받아
  # 2. 패딩 마스크를 모델 아웃풋과 곱할 수 있도록 차원을 바꾸고 형변환
  # 3. 모델 아웃풋과 패딩 마스크를 곱해 패딩 토큰을 0으로 만들고
  # 4. 모델 아웃풋을 모두 더한다
  # 5. 평균을 구하기 위해 패딩 마스크를 더해 단어 개수를 뽑고 (0으로 나누지 않기 위해서 최솟값 부여)
  # 6. 더한 출력 임베딩의 합을 단어 개수로 나눈다

# **코드로 살펴보는 최대 모드**

In [ ]:
def max_pooling(model_output, attention_mask):
  token_embeddings = model_output[0]
  input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()

  # 텐서 조건 인덱싱
  # x = torch.tensor([10, 20, 30, 40, 50])
  # x[x > 25] = -1  # 25보다 큰 값만 -1로 변경
  # print(x)

  # token_embeddings와 input_mask_expanded의 차원이 같기 때문에
  # input_mask_expanded를 순회하면서 0인 인덱스를 찾아 token_embeddings에 적용
  # 이후 -1e9 할당
  # 즉 패딩 토큰이 최댓값이 절대 될 수 없도록 변경
  token_embeddings[input_mask_expanded == 0] = -1e9

  # 최댓값 반환
  # [0] = 최댓값
  # [1] = 최댓값이 있는 위치
  return torch.max(token_embeddings, 1)[0]